In [3]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import struct

解码文件的代码http://www.manongjc.com/article/5265.html

In [4]:
train_labels_path= r'D:\data_ML\minist\train-labels.idx1-ubyte'
with open(train_labels_path, 'rb') as lpath:
    # '>' denotes bigedian
    # 'I' denotes unsigned char
    magic, n = struct.unpack('>II', lpath.read(8))
    #loaded = np.fromfile(lpath, dtype = np.uint8)
    train_labels = np.fromfile(lpath, dtype = np.uint8).astype(np.float)

In [5]:
test_images_path= r'D:\data_ML\minist\train-images.idx3-ubyte'
with open(test_images_path , 'rb') as f:
    magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
    loaded = np.fromfile(test_images_path, dtype = np.uint8)
    # images start from the 16th bytes
    train_images = loaded[16:].reshape(len(train_labels), 784).astype(np.float)

# 数据模块

In [4]:
class DiabetesDataset(Dataset):
    def __init__(self, x_, y_):
        self.len = x_.shape[0]
        self.x_data = torch.from_numpy(x_)
        # 这部很关键， 特征类型要求float类型
        self.x_data = self.x_data.float()
        self.y_data = torch.LongTensor(y_)

    def __getitem__(self, item):
        return self.x_data[item], self.y_data[item]

    def __len__(self):
        return self.len

In [15]:
dataset = DiabetesDataset(train_images, train_labels)
train_loader = DataLoader(dataset=dataset, batch_size=200, shuffle=True)

In [16]:
len(train_loader)

300

# 网络结构

In [6]:
#网络结构
class Lenet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.maxpool_1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv_2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=0)
        self.maxpool_2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0) 
        
        self.fc1 = nn.Linear(32*4*4,28)
        self.fc2 = nn.Linear(28,10)
    
    def forward(self, x):
        x = x.reshape(x.shape[0],1,28,28)
        out = self.conv_1(x)
        out = self.maxpool_1(out)
        out = self.conv_2(out)
        out = self.maxpool_2(out)
        # 运行时候后面线性层注释，然后调试看下卷积操作后维度时候多少，然后设计最后的线性层
        out = out.reshape(out.shape[0],32*4*4)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

# 测试下维度

In [12]:
a = train_images[0:2].reshape(2,1,28,28)
net = Lenet5()
net(torch.from_numpy(a).float()).shape

torch.Size([2, 10])

# 模型训练

In [17]:
device = torch.device('cuda')
model = Lenet5().to(device)
#损失函数
criterion = nn.CrossEntropyLoss(reduction='mean')
#优化算法
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
total_step = len(train_loader)
num_epochs = 50
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        # Forward pass
        outputs = model(x)
        #扁平化成一维的
        y = y.squeeze()
        loss = criterion(outputs, y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #输出中间信息
        if i % 100 == 0 :
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/50], Step [1/300], Loss: 0.0224
Epoch [1/50], Step [101/300], Loss: 0.0710
Epoch [1/50], Step [201/300], Loss: 0.0294
Epoch [2/50], Step [1/300], Loss: 0.0032
Epoch [2/50], Step [101/300], Loss: 0.0333
Epoch [2/50], Step [201/300], Loss: 0.0745
Epoch [3/50], Step [1/300], Loss: 0.0302
Epoch [3/50], Step [101/300], Loss: 0.0250
Epoch [3/50], Step [201/300], Loss: 0.0437
Epoch [4/50], Step [1/300], Loss: 0.0119
Epoch [4/50], Step [101/300], Loss: 0.0011
Epoch [4/50], Step [201/300], Loss: 0.0285
Epoch [5/50], Step [1/300], Loss: 0.0130
Epoch [5/50], Step [101/300], Loss: 0.0187
Epoch [5/50], Step [201/300], Loss: 0.0188
Epoch [6/50], Step [1/300], Loss: 0.0206
Epoch [6/50], Step [101/300], Loss: 0.0308
Epoch [6/50], Step [201/300], Loss: 0.0033
Epoch [7/50], Step [1/300], Loss: 0.1014
Epoch [7/50], Step [101/300], Loss: 0.0068
Epoch [7/50], Step [201/300], Loss: 0.0009
Epoch [8/50], Step [1/300], Loss: 0.0072
Epoch [8/50], Step [101/300], Loss: 0.0056
Epoch [8/50], Step [201/300

# 保存模型

In [48]:
# 保存模型
path = './ft.pth'
torch.save(model.state_dict(), path)

# 预测

In [16]:
# 加载模型,模型和数据还是要放到cpu上去，全量的数据放gpu显存放不下
model = Lenet5().to(torch.device('cpu'))
model.load_state_dict(torch.load('./ft.pth'))
#必须调用model.eval()将dropout和batch normalization层设置为测试模式，不然会导致不一致的结果。
model.eval()

Lenet5(
  (conv_1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (maxpool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (maxpool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=28, bias=True)
  (fc2): Linear(in_features=28, out_features=10, bias=True)
)

In [22]:
predicted = model(torch.LongTensor(train_images).float().to(torch.device('cpu')))

In [23]:
predicted_class = predicted.argmax(dim=1)

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
accuracy_score(predicted_class.cpu().detach().numpy(),train_labels)

0.9963666666666666

In [26]:
predicted_class.shape

torch.Size([60000])